In [2]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
batch_size = 32 # arbitrary
num_classes = 2 # class_1 = tom, class_2 = jerry
epochs = 20 # how many passes forward and backwards

imageSize = (1280, 720)

train, test  = tf.keras.preprocessing.image_dataset_from_directory(
    "./tom_and_jerry",
    validation_split=.2,
    subset="both",
    seed=1234,
    image_size=imageSize,
    label_mode="binary",
    color_mode="rgb",
    batch_size=32
)

Found 3170 files belonging to 2 classes.
Using 2536 files for training.
Using 634 files for validation.


In [4]:
print(train)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 1280, 720, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>


In [24]:
for image_batch, labels_batch in train:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 1280, 720, 3)
(32, 1)


In [32]:
class_names = train.class_names
print(class_names)

['jerry', 'tom']


In [27]:
normalization_layer = layers.Rescaling(1./255)

In [29]:
normalized_ds = train.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.99983156


In [34]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(1280, 720, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 1280, 720, 3)      0         
                                                                 
 conv2d (Conv2D)             (None, 1280, 720, 16)     448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 640, 360, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 640, 360, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 320, 180, 32)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 320, 180, 64)     

In [36]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [38]:
epochs=10
history = model.fit(
  train,  
  validation_data=test,
  epochs=epochs
)

Epoch 1/10
19/80 [======>.......................] - ETA: 14:23 - loss: 4.3527 - accuracy: 0.5280

KeyboardInterrupt: 